In [1]:
using ForwardDiff, pulse_input_DDM

In [2]:
B,n,dt,vara,lambda = 15.,53,1e-2,40.,-1.

(15.0, 53, 0.01, 40.0, -1.0)

In [3]:
xc, dx, = pulse_input_DDM.bins(B,n)

([-15.2941, -14.7059, -14.1176, -13.5294, -12.9412, -12.3529, -11.7647, -11.1765, -10.5882, -10.0  …  10.0, 10.5882, 11.1765, 11.7647, 12.3529, 12.9412, 13.5294, 14.1176, 14.7059, 15.2941], 0.5882352941176471, [-15.5882, -15.0, -14.4118, -13.8235, -13.2353, -12.6471, -12.0588, -11.4706, -10.8824, -10.2941  …  10.2941, 10.8824, 11.4706, 12.0588, 12.6471, 13.2353, 13.8235, 14.4118, 15.0, 15.5882])

In [4]:
function M1!(x::Vector{TT}, h::Union{UU,Float64}, dx::UU,
        xc::Vector{UU}, n::Int, dt::Float64) where {TT,UU <: Any}
    
    F = zeros(TT,n,n)
    vara,lambda = x
    
    F[1,1] = one(TT); F[n,n] = one(TT); F[:,2:n-1] = zeros(TT,n,n-2)

    #changed 2/17 to keep to less than 1000 bins, haven't checked how that effects returned results
    ndeltas = max(70,ceil(Int, 10. *sqrt(vara)/dx))   
    #ndeltas = 70 + (1000 - 70) * ceil(Int, 0.5*(1+tanh(10. *sqrt(vara)/dx)))
    ndeltas > 1000 ? (ndeltas = 1000; @warn "using lots of bins!";) : nothing

    (ndeltas > 1e3 && h == zero(TT)) ? (println(vara); println(dx); println(ndeltas)) : nothing

    #deltas = collect(-ndeltas:ndeltas) * (5.*sqrt(vara))/ndeltas;
    #ps = broadcast(exp, broadcast(/, -broadcast(^, deltas,2), 2.*vara)); ps = ps/sum(ps);
    
    deltaidx = collect(-ndeltas:ndeltas);
    deltas = deltaidx * (5. *sqrt(vara))/ndeltas;
    ps = exp.(-0.5 * (5*deltaidx./ndeltas).^2); ps = ps/sum(ps);
    
    @inbounds for j = 2:n-1

        abs(lambda) < 1e-150 ? mu = xc[j] + h * dt : mu = exp(lambda*dt)*(xc[j] + h/lambda) - h/lambda
        
        #now we're going to look over all the slices of the gaussian
        for k = 1:2*ndeltas+1

            s = mu + deltas[k]

            if s <= xc[1]

                F[1,j] += ps[k];

            elseif s >= xc[n]

                F[n,j] += ps[k];

            else

                if xc[1] < s && xc[2] > s

                    lp,hp = 1,2;

                elseif xc[n-1] < s && xc[n] > s

                    lp,hp = n-1,n;

                else

                    hp,lp = ceil(Int, (s-xc[2])/dx) + 2, floor(Int, (s-xc[2])/dx) + 2;

                end

                if (hp == lp)

                    F[lp,j] += ps[k];

                else

                    dd = xc[hp] - xc[lp];
                    F[hp,j] += ps[k]*(s-xc[lp])/dd;
                    F[lp,j] += ps[k]*(xc[hp]-s)/dd;

                end

            end

        end

    end
    
    sum(F)

end

M1! (generic function with 1 method)

In [5]:
myconvert(::Type{T}, x::ForwardDiff.Dual) where {T} = T(x.value)

myconvert (generic function with 1 method)

In [6]:
function M2!(x::Vector{TT}, h::Union{UU,Float64}, dx::UU,
        xc::Vector{UU}, n::Int, dt::Float64) where {TT,UU <: Any}
    
    F = zeros(TT,n,n)
    vara,lambda = x
    
    F[1,1] = one(TT); F[n,n] = one(TT); F[:,2:n-1] = zeros(TT,n,n-2)
    
    if typeof(vara) == ForwardDiff.Dual{Nothing,Float64,0}
        sigma2_sbin = myconvert(Float64, vara)
    else
        sigma2_sbin = vara
    end

    #changed 2/17 to keep to less than 1000 bins, haven't checked how that effects returned results
    ndeltas = max(70,ceil(Int, 10. *sqrt(sigma2_sbin)/dx))   
    #ndeltas = 70 + (1000 - 70) * ceil(Int, 0.5*(1+tanh(10. *sqrt(vara)/dx)))
    #ndeltas > 1000 ? (ndeltas = 1000; @warn "using lots of bins!";) : nothing

    #(ndeltas > 1e3 && h == zero(TT)) ? (println(vara); println(dx); println(ndeltas)) : nothing

    deltas = collect(-ndeltas:ndeltas) * (5. *sqrt(sigma2_sbin))/ndeltas;
    #ps = broadcast(exp, broadcast(/, -broadcast(^, deltas,2), 2. *vara)); ps = ps/sum(ps);
    ps = exp.(-deltas.^2 / (2. * vara)); ps = ps/sum(ps);
    
    #deltaidx = collect(-ndeltas:ndeltas);
    #deltas = deltaidx * (5. *sqrt(vara))/ndeltas;
    #ps = exp.(-0.5 * (5*deltaidx./ndeltas).^2); ps = ps/sum(ps);
    
    @inbounds for j = 2:n-1

        abs(lambda) < 1e-150 ? mu = xc[j] + h * dt : mu = exp(lambda*dt)*(xc[j] + h/lambda) - h/lambda
        
        #now we're going to look over all the slices of the gaussian
        for k = 1:2*ndeltas+1

            s = mu + deltas[k]

            if s <= xc[1]

                F[1,j] += ps[k];

            elseif s >= xc[n]

                F[n,j] += ps[k];

            else

                if xc[1] < s && xc[2] > s

                    lp,hp = 1,2;

                elseif xc[n-1] < s && xc[n] > s

                    lp,hp = n-1,n;

                else

                    hp,lp = ceil(Int, (s-xc[2])/dx) + 2, floor(Int, (s-xc[2])/dx) + 2;

                end

                if (hp == lp)

                    F[lp,j] += ps[k];

                else

                    dd = xc[hp] - xc[lp];
                    F[hp,j] += ps[k]*(s-xc[lp])/dd;
                    F[lp,j] += ps[k]*(xc[hp]-s)/dd;

                end

            end

        end

    end
    
    sum(F)

end

M2! (generic function with 1 method)

In [7]:
ForwardDiff.gradient(x-> M1!(x,0.,dx,xc, n, dt), [vara,lambda])

2-element Array{Float64,1}:
 -9.269928574751063e-18 
  1.6479873021779667e-17

In [8]:
ForwardDiff.gradient(x-> M2!(x,0.,dx,xc, n, dt), [vara,lambda])

2-element Array{Float64,1}:
 -1.3877787807814457e-17
 -8.673617379884035e-19 

In [9]:
M1!([vara,lambda],0.,dx,xc, n, dt)

53.0

In [10]:
M2!([vara,lambda],0.,dx,xc, n, dt)

53.0

In [65]:
typeof(ForwardDiff.Dual.(1.2)) == ForwardDiff.Dual{Nothing,Float64,0}

true